# Dates and Times

Being able to handle dates and times, particularly for time series applications, is a critical part of data-cleaning. It is easy to let subtle bugs creep in due to parsing dates and times incorrectly or not accounting for timezones. On rare occasion, you may encounter datasets coming from a system that bravely uses custom time types, like 27-hour clocks (yes, this happened to me!). The point is, working with dates and times can be messy so we will learn some practical strategies here. 



In [ ]:
import pandas as pd 
import numpy as np 

We are going to load a dataframe from Github that contains 5 columns of dates and times, all representing the same value but in 5 different formats. 

In [ ]:
url = 'https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/timeseries/datetime_formatting.csv'

df = pd.read_csv(url)
df

Let's extract one of the columns and look at its datatypes. Noteice that it is a `dtype` of `object`, not a `datetime64` as we would want. 

In [ ]:
df['ORDER_DATE_TM1']

## Implicit Datetime Conversion

If you want to do any useful calendar or time-based logic on these values, you will have to convert them to a different data type. One way to do this is to call Pandas' `to_datetime()` function on that column. Pandas will then do its best to parse the date for that series. 

In [ ]:
parsed_col = pd.to_datetime(df['ORDER_DATE_TM1'])
parsed_col

We can use calendar-based logic to extract properties, like the day of week. 

In [ ]:
parsed_col.dt.dayofweek

If you already know which columns you want to format as dates/times beforehand, you can pass the `parse_dates` parameter to the `read_csv()` function with a list of column names to expect dates/times form. Let's parse all the dates using this approach and analyze the result. For brevity, let's only look at the first three results. 

In [ ]:
df_parsed = pd.read_csv(url, 
            parse_dates=['ORDER_DATE1','ORDER_DATE2','ORDER_DATE_TM1','ORDER_DATE_TM2','ORDER_DATE_TM3'])

df_parsed.head(3)

Are there any errors? As a matter of fact, the `ORDER_DATE_TM2` was parsed almost completely in error! For example, the second record really has values representing a datetime of `2022-09-25 20:16:00` but the `ORDER_DATE_TM2` was wrongly parsed as `2025-09-22 20:16:00`! What happened? 

Well let's look at the original value. As a matter of fact, let's sample the first three records and analyze what's going on.

In [ ]:
df.head(3)

What seems to be happening with `ORDER_DATE_TM2` is it is confusing the day and month. Conventionally one might expect a format like `22-Sep-25 8:16 PM` to have the day `22` to be leading, and this is what Pandas assumed. However, a sadistic developer decided to arbitrate their own convention and record the year in that place instead, and put the day `25` after the month. 

This explains why the first record `22-Jan-22 4:08PM` happened to be parsed correctly, as the year and day of month were exactly the same. 

To handle this, we will need to do explicit conversion.

## Explicit Datetime Conversion

Study the datetime formatting conventions for Python here. 

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

`strftime()` and `strptime()` are used to write a datetime to a formatting string, and parse a datetime from a formatted string respectively. The format codes come from the standard C conventions. Here are a few common ones, many of which we will use in this notebook. Refer to the link above to see all format codes. 

| Symbol | Description             | Parse Example: 2021-01-13 2:34PM |
|--------|-------------------------|----------------------------------|
| `%y`     | 2-digit year            | 21                               |
| `%m`     | 2-digit month           | 01                               |
| `%d`     | 2-digit day of month    | 13                               |
| `%b`     | 3-letter month          | Jan                              |
| `%I`     | Hour for 12-hour clock  | 2                                |
| `%H`     | Hour for 24-hour clock  | 14                               |
| `%M`     | 2-digit minute          | 34                               |
| `%p`     | AM/PM for 12-hour clock | PM                               |
| `%S`     | 2-digit seconds         | 00                               |
| `%f`     | Microseconds            | 000000                           |
| `%a`     | 3-letter weekday        | Wed                              |
| `%A`     | Full name weekday       | Wednesday                        |

For our problmatic `ORDER_DATE_TM2` column, we need `%y` to get a two-digit year, a `%b` for the three-letter name of the month, and `%d` for the day of month. For the time we use `%I` for the 12-hour clock hour, `%M` for the minute, and `%p` for the `AM/PM`. We will repair the `ORDER_DATE_TM2` parsing by passing it to the `to_datetime()` function along with the `format` parameter, and then assign this back to the parsed dataframe. 

In [ ]:
df_parsed['ORDER_DATE_TM2'] = pd.to_datetime(df['ORDER_DATE_TM2'], format='%y-%b-%d %I:%M %p')
df_parsed

Much better! Now all the dates are working. We can verify this by counting the number of rows where there is only one unique value between `ORDER_DATE_TM1`,`ORDER_DATE_TM2`, and `ORDER_DATE_TM3`. Sure enough, all 1190 rows have exactly one unique datetime value in each row. 

In [ ]:
np.sum(df_parsed.loc[:,['ORDER_DATE_TM1','ORDER_DATE_TM2','ORDER_DATE_TM3']].nunique(axis=1) == 1)

We can do this same verification for the two plain dates columns to make sure they were parsed correctly too. 

In [ ]:
np.sum(df_parsed.loc[:,['ORDER_DATE1','ORDER_DATE2']].nunique(axis=1) == 1)

## Filtering on Datetimes

Let's consolidate our dataframe and rename the columns to just `ORDER_DATE` and `ORDER_DATE_TM`

In [ ]:
df = df_parsed[['RECORD_ID','ORDER_DATE1','ORDER_DATE_TM1']] \
    .rename(columns={"ORDER_DATE1": "ORDER_DATE", "ORDER_DATE_TM1": "ORDER_DATE_TM"})

df

Let's say we wanted to get all records where the day of week is Tuesday. In the `dt.dayofweek` property, `0` is going to be Monday and `7` is going to be Sunday. Numerically, this will make Tuesday be a `1`. We can specify this as a condtion and return all records that fall on Tuesday. 

In [ ]:
condition = df['ORDER_DATE'].dt.dayofweek == 1
df[condition]

You can also quickly filter dates and times using a string format. 

In [ ]:
df[df['ORDER_DATE'] >= '2022-06-01']

In [ ]:
df[('2022-06-01 12:35PM' <= df['ORDER_DATE_TM']) & (df['ORDER_DATE_TM'] <= '2022-06-05 8:05PM')]

You can also do more explicit datetime conversions for the start and end bounds. 

In [ ]:
start = pd.to_datetime('2022-06-01 12:35PM', format='%Y-%m-%d %I:%M%p')
end = pd.to_datetime('2022-06-05 8:05PM', format='%Y-%m-%d %I:%M%p')

df[(start <= df['ORDER_DATE_TM']) & (df['ORDER_DATE_TM'] <= end)]

## Dealing with Timezones

Likely one of the biggest headaches you can encounter in data cleaning when it comes to dates and times is timezone conversions. It is ideal if datetimes are stored in **coordinated universal times or UTC**, which is an internationally agreed standard for storing datetimes. If times need to be converted locally, then **offsets** are used to express that local time based off the UTC time. This sounds easier than it actually is, because regional laws around the world evolve and change offsets especially due to daylight savings time. 

Thankfully there is a convenient library called `pytz` that Pandas already depends on. It will take care of timezone offsets as well as daylight savings time, even capturing DST laws changed in the past! Let's import it and look at the common timezones. 

In [ ]:
import pytz

pytz.common_timezones

Whoa! That's a lot. Let's say we want to localize to Arizona time in the United States. Arizona is kind of special because it does not recognize daylight savings time like the rest of the United States. It gets enough sunlight year round! 

We will look it up by name and save it to a variable. 

In [ ]:
tz = pytz.timezone('US/Arizona')
tz

Now let's look at our `ORDER_DATE_TM` column in our dataframe. Notice that there is no information about the timezone, making it **timezone naive**.

In [ ]:
if df['ORDER_DATE_TM'].dt.tz is None:
    print("TZ NAIVE")
else:
    df['ORDER_DATE_TM'].dt.tz

Let's say these dates were actually recorded in Arizona local time. We can make `ORDER_DATE_TM` timezone-aware by calling the `tz_localize()` function and specifying they were recorded in `US/Arizona` time. 

In [ ]:
df['ORDER_DATE_TM'] = df['ORDER_DATE_TM'].dt.tz_localize('US/Arizona')
df['ORDER_DATE_TM']

Notice how the datatype of `ORDER_DATE_TM` changes to `datetime64[ns, US/Arizona]`, making it no longer timezone naive and now associated with Arizona time. The `-07:00` part of the value indicates the offset from UTC. 

We are now free to convert it to different timezones. Let's say we wanted to add an additional column `ORDER_DATE_TM_CST` showing the datetime in US/Central time. We can use `tz_convert` to perform this conversion. Notice below how central time is sometimes 2 hours ahead of Arizona, but also sometimes 1 hour ahead. This is because Arizona does not observe daylight savings time but central time does. 

In [ ]:
df['ORDER_DATE_TM_CST'] = df['ORDER_DATE_TM'].dt.tz_convert('US/Central')
df

Finally, we can of course convert the date to `UTC` which we will save to a column `ORDER_DATE_TM_UTC`. Notice the `+00:00` offset as UTC is the baseline with no offset.

In [ ]:
df['ORDER_DATE_TM_UTC'] = df['ORDER_DATE_TM'].dt.tz_convert('UTC')
df

Isn't it nice how much work the `tz` library did for you? It is encouraged to use it as it will maintain that complex database of timezones, offsets, daylight savings, and historical changes to daylight savings laws for you. 

## EXERCISE

A dataframe of two columns and two records is shown below. Complete the code below by replacing the question marks `?` so that `ORDER_DATE_TM` is localized to `US/Pacific`. Then add a new column `ORDER_DATE_TM_PARIS` that shows the equivalent time in `Europe/Paris`.  

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "ORDER_ID": [1, 2], 
    "ORDER_DATE_TM": [pd.to_datetime('2023-01-05 7:05 PM'), pd.to_datetime('2023-01-06 8:15 AM')]
})

# localize to US/Pacific
df["ORDER_DATE_TM"] = df["ORDER_DATE_TM"].?

# convert to Europe/Paris 
df["ORDER_DATE_TM_PARIS"] = ?

df

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "ORDER_ID": [1, 2], 
    "ORDER_DATE_TM": [pd.to_datetime('2023-01-05 7:05 PM'), pd.to_datetime('2023-01-06 8:15 AM')]
})

# localize to US/Pacific
df["ORDER_DATE_TM"] = df["ORDER_DATE_TM"].dt.tz_localize('US/Pacific')

# convert to Europe/Paris 
df["ORDER_DATE_TM_PARIS"] = df["ORDER_DATE_TM"].dt.tz_convert('Europe/Paris')

df